In [1]:
!pip install transformers sentencepiece pdfminer.six spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
from google.colab import files


uploaded = files.upload()


Saving Intelligent_Signal_Processing_Finalterm_Coursework - v3.pdf to Intelligent_Signal_Processing_Finalterm_Coursework - v3.pdf


In [6]:

# -----------------------------
# 1. Imports
# -----------------------------
from transformers import pipeline
from pdfminer.high_level import extract_text
import spacy
import re
from collections import defaultdict

# -----------------------------
# 2. Load Models
# -----------------------------
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)
nlp = spacy.load("en_core_web_sm")

# -----------------------------
# 3. Helpers
# -----------------------------
def extract_text_from_pdf(file_path):
    """Extract raw text from PDF"""
    return extract_text(file_path)

def split_text(text, max_chunk_size=1000):
    """Split long text into smaller chunks for summarization"""
    sentences = re.split(r'(?<=[\.\n])', text)
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

def summarize_text(text):
    """Summarize long text by breaking it into smaller chunks"""
    chunks = split_text(text)
    summaries = []
    for chunk in chunks:
        try:
            summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)
            summaries.append(summary[0]['summary_text'])
        except Exception as e:
            print("⚠️ Chunk skipped due to error:", e)
    return "\n\n".join(summaries)

def extract_entities(text):
    """Extract named entities using spaCy"""
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

def format_entities(entities):
    """Group and format entities by label"""
    grouped = defaultdict(set)
    for text, label in entities:
        grouped[label].add(text)
    return grouped

# -----------------------------
# 4. Main Execution
# -----------------------------
file_path = "Intelligent_Signal_Processing_Finalterm_Coursework - v3.pdf"
text = extract_text_from_pdf(file_path)
# -----------------------------
# 5. Display Output Clearly
# -----------------------------
# ✅ Print a clean sample of extracted text
print("📄 Extracted Text Sample:\n" + "-"*30)
print(text[:1000])
print("-"*30 + "\n")

# 📝 Summarize
summary = summarize_text(text)
print("📝 SUMMARY:\n" + "-"*30)
for i, chunk in enumerate(summary.split("\n\n")):
    print(f"🔹 Paragraph {i+1}:\n{chunk.strip()}\n")
print("-"*30 + "\n")

# 🔍 Named Entities
entities = extract_entities(text)
grouped_entities = format_entities(entities)

print("🔍 NAMED ENTITIES (NER):\n" + "-"*30)
for label, items in grouped_entities.items():
    print(f"\n🔸 {label}:")
    for item in sorted(items):
        print(f"   - {item}")
print("-"*30)


Device set to use cpu


📄 Extracted Text Sample:
------------------------------
Intelligent Signal Processing  

Module: Intelligent Signal Processing 

Coursework: end-of-term 

Submission Deadline: Please refer to the Coursera. 

•  Please Note: You are permitted to upload your Coursework in the final submission area as many 
times  as  you  like  before  the  deadline.  You  will  receive  a  similarity/originality  score  which 
represents what the Turnitin system identifies as work similar to another source. The originality 
score can take over 24 hours to generate, especially at busy times e.g. submission deadline.   

• 

If you upload the wrong version of your Coursework, you are able to upload the correct version of 
your Coursework via the same submission area. You simply need to click on the  'submit paper' 
button again and submit your new version before the deadline. 

In  doing  so,  this  will  delete  the  previous  version  which  you  submitted  and  your  new  updated 
version will replace 

Your max_length is set to 150, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


📝 SUMMARY:
------------------------------
🔹 Paragraph 1:
You are permitted to upload your Coursework in the final submission area as many times as you like before the deadline. You will receive a similarity/originality score which represents what the Turnitin system identifies as work similar to another source. The originality  score can take over 24 hours to generate, especially at busy times.

🔹 Paragraph 2:
The End of term assignment for Intelligent Signal Processing consists of a series of three individual exercises. These exercises cover the last five topics of the course. The total word count for each exercise should not exceed 2500 words.

🔹 Paragraph 3:
The exercises are strongly based on the subjects covered during the course, but also invite the student for further investigation. There are three exercises, and you are required to submit the following files: One PDF file containing a report for all exercises. One ZIP file (containing sub-folders that include files for each exe

In [7]:
 !pip install transformers


In [8]:

from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [9]:
context = text  # Or use summary for faster QA


In [18]:
question = "how many exercise"
result = qa_pipeline(question=question, context=context)
print(f"Q: {question}\nA: {result['answer']}")


Q: how many exercise
A: 3
